In [1039]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
from datetime import datetime
from datetime import timedelta

In [1040]:
# added OS and browser info
def readGoogleAnalytics(filename):
    header = ['category', 'date', 'OS', 'browser', 'version', 'type', 'expression', 'localization', 'local time', 'remote time', 'total time', 'status']
    df = pd.DataFrame(columns=header)
    i = 0
    with open(filename, "r") as csvfile:
        reader = csv.DictReader(filter(lambda row: row[0] != '#', csvfile))
        for rows in reader:
            row = list(rows.values())[0]
            words = row[0].split(sep=',')
            if (len(header) == len((words))):
                df.loc[i] = words
                i = i + 1
    return df

# Google analytics dataframe

In [1041]:
df = readGoogleAnalytics('gace 160913.csv')
df.head()

,category,date,OS,browser,version,type,expression,localization,local time,remote time,total time,status
0,column name,2016/08/25 21:45:58,MacIntel,Chrome,52.0.2743.116,COLUMN_NAMES,"'""hello""' at 0",local,0,0,0,OK
1,doc,2016/08/31 16:09:19,Intel Mac OS X 10.11.6,Chrome,52.0.2743.116,VALUE_DOC,abs,remote,0,176,176,OK
2,doc,2016/08/31 16:09:20,Intel Mac OS X 10.11.6,Chrome,52.0.2743.116,VALUE_DOC,all,remote,0,171,171,OK
3,doc,2016/08/31 16:09:20,Intel Mac OS X 10.11.6,Chrome,52.0.2743.116,VALUE_DOC,any,remote,0,163,163,OK
4,doc,2016/08/31 16:09:22,Intel Mac OS X 10.11.6,Chrome,52.0.2743.116,VALUE_DOC,ArithmeticError,remote,0,164,164,OK


In [1042]:
x = df["date"]
t = []
for i in range(len(x)):
    t.append(datetime.strptime(x[i][1:-9], "%Y/%m/%d"))
df['day'] = t
df.head()

,category,date,OS,browser,version,type,expression,localization,local time,remote time,total time,status,day
0,column name,2016/08/25 21:45:58,MacIntel,Chrome,52.0.2743.116,COLUMN_NAMES,"'""hello""' at 0",local,0,0,0,OK,2016-08-25
1,doc,2016/08/31 16:09:19,Intel Mac OS X 10.11.6,Chrome,52.0.2743.116,VALUE_DOC,abs,remote,0,176,176,OK,2016-08-31
2,doc,2016/08/31 16:09:20,Intel Mac OS X 10.11.6,Chrome,52.0.2743.116,VALUE_DOC,all,remote,0,171,171,OK,2016-08-31
3,doc,2016/08/31 16:09:20,Intel Mac OS X 10.11.6,Chrome,52.0.2743.116,VALUE_DOC,any,remote,0,163,163,OK,2016-08-31
4,doc,2016/08/31 16:09:22,Intel Mac OS X 10.11.6,Chrome,52.0.2743.116,VALUE_DOC,ArithmeticError,remote,0,164,164,OK,2016-08-31


# Pivoting data

In [1043]:
# data casting
df["local time"] = df["local time"].astype(int)
df["remote time"] = df["remote time"].astype(int)
df["total time"] = df["total time"].astype(int)
# define quantiles
def q05(x):
    if np.count_nonzero(x)>0:
        return np.percentile(x[x>0],5)
    else:
        return 0
def q95(x):
    if np.count_nonzero(x)>0:
        return np.percentile(x[x>0],95)
    else:
        return 0
# define functions for non-zero statistics computation
def nonzeromean(x):
    return np.mean(x[x>0])
def nonzerostd(x):
    return np.std(x[x>0])
def nonzeromin(x):
    return np.min(x[x>0])
def nonzeromax(x):
    return np.max(x[x>0])

# pivot the data
pt = pd.pivot_table(df, index=['expression','localization'], values=["total time"], aggfunc=[nonzeromean, nonzerostd, nonzeromin, nonzeromax, np.count_nonzero, q05, q95])
pt

,,nonzeromean,nonzerostd,nonzeromin,nonzeromax,count_nonzero,q05,q95
,,total time,total time,total time,total time,total time,total time,total time
expression,localization,,,,,,,
"""hello""",remote,5508.000000,0.000000,5508.0,5508.0,1,5508.00,5508.00
"'""hello""' at 0",local,NaN,NaN,NaN,NaN,0,0.00,0.00
ArithmeticError,remote,164.000000,0.000000,164.0,164.0,1,164.00,164.00
a,combined,1164.000000,0.000000,1164.0,1164.0,1,1164.00,1164.00
abs,remote,176.000000,0.000000,176.0,176.0,1,176.00,176.00
aes,local,168.000000,0.000000,168.0,168.0,1,168.00,168.00
all,remote,171.000000,0.000000,171.0,171.0,1,171.00,171.00
any,remote,163.000000,0.000000,163.0,163.0,1,163.00,163.00


In [1044]:
pt = pd.pivot_table(df, index=['expression','localization'], values=["total time"], aggfunc=[nonzeromean, nonzerostd, nonzeromin, nonzeromax, np.count_nonzero, q05, q95])
pt.query('expression == [" np"]')

nonzeromean    nonzerostd nonzeromin nonzeromax  \
                           total time    total time total time total time   
expression localization                                                     
 np         combined     11189.887097  12877.917853      167.0    45586.0   
            local          192.304348    473.971604       36.0     3715.0   

                        count_nonzero        q05        q95  
                           total time total time total time  
expression localization                                      
 np         combined               62     187.25    39946.8  
            local                  69      43.40      595.0

In [1045]:
expressions = df['expression']
expr = [expression for expression in expressions if " np" in expression]
expr = np.unique(expr).tolist()
pd.pivot_table(df[df.expression.isin(expr)], index=['expression','localization'], values=["total time"], aggfunc=[nonzeromean, nonzerostd, nonzeromin, nonzeromax, np.count_nonzero, q05, q95])

nonzeromean    nonzerostd nonzeromin  \
                                  total time    total time total time   
expression        localization                                          
 np                combined     11189.887097  12877.917853        167   
                   local          192.304348    473.971604         36   
 np._NoValue       remote         139.000000      0.000000        139   
 np._import_tools  remote         141.000000      0.000000        141   
 np._mat           remote         141.000000      0.000000        141   
 np.abs            remote         142.000000      0.000000        142   
 np.absolute       remote         143.000000      0.000000        143   
 np.random         combined      5594.000000   7416.315932         79   
                   remote       16233.096774  25026.844379         78   

                               nonzeromax count_nonzero        q05        q95  
                               total time    total time total time total time  
expression        localization                                                 
 np                combined         45586            62     187.25   39946.80  
                   local             3715            69      43.40     595.00  
 np._NoValue       remote             139             1     139.00     139.00  
 np._import_tools  remote             141             1     141.00     141.00  
 np._mat           remote             141             1     141.00     141.00  
 np.abs            remote             142             1     142.00     142.00  
 np.absolute       remote             143             1     143.00     143.00  
 np.random         combined         20281             6     104.00   17680.75  
                   remote           97115            31     178.50   70981.50

In [1046]:
expressions = pt.index.levels[0]
np_expr = [expression for expression in expressions if " np" in expression]
np_expr

[' np',
 ' np._NoValue',
 ' np._import_tools',
 ' np._mat',
 ' np.abs',
 ' np.absolute',
 ' np.random']

In [1076]:
# pivot the data for pie chart
#pt_pie = pd.pivot_table(df, index=['expression'], values=["local time"], aggfunc=[np.count_nonzero])
#pt_pie = pd.pivot_table(df, index=['expression'], values=["remote time"], aggfunc=[np.count_nonzero])
pt_pie = pd.pivot_table(df, index=['expression'], values=["local time"], aggfunc=[sum])
#pt_pie = pd.pivot_table(df, index=['expression'], values=["remote time"], aggfunc=[sum])
pt_pie = pt_pie.dropna() # exclude NA's
# pie chart
from pylab import *

labels = pt_pie.index
values = pt_pie.dropna().values[0:,0]
explode = [0.05]*len(values)

values, labels = zip(*sorted(zip(values, labels),reverse=True))

values = np.asarray(values)
fracs = 100.0*values/values.sum() 
leg_labels = ['{0} - {1:1.1f} %'.format(i,j) for i,j in zip(labels, fracs)]

# truncate output
B = 10
values = values[0:B]
labels = labels[0:B]
explode = explode[0:B]
leg_labels =leg_labels[0:B]
fracs = fracs[0:B]/100

pie=pie(fracs,labels=labels,autopct='%1.1f%%',pctdistance=1.1,labeldistance=1.2,radius=1.2,explode=explode,shadow=True)
legend(pie[0], leg_labels, loc="upper left").get_frame().set_alpha(0.5)
#legend(pie[0], leg_labels, loc="lower right").get_frame().set_alpha(0.5)
subplots_adjust(right=0.8,top=0.8)
#subplots_adjust(left=0.25,top=0.8)
#title('Top 10 most frequent completion expressions (local)',y=1.15)
#title('Top 10 most frequent completion expressions (remote)',y=1.15)
title('Top 10 most time-consuming completion expressions (local)',y=1.15)
#title('Top 10 most time-consuming completion expressions (remote)',y=1.15)
show()

In [1048]:
pt = pd.pivot_table(df, index=['day'], values=["total time"], aggfunc=[nonzeromean, nonzerostd, nonzeromin, nonzeromax, np.count_nonzero, q05, q95])
pt

,nonzeromean,nonzerostd,nonzeromin,nonzeromax,count_nonzero,q05,q95
,total time,total time,total time,total time,total time,total time,total time
day,,,,,,,
2016-08-25,9919.914894,14113.056585,25,45586,47,34.00,40737.80
2016-08-26,7917.750000,20626.355894,57,97115,44,116.15,30598.05
2016-08-27,1642.533333,2208.123875,55,6792,15,57.10,6131.20
2016-08-28,196.000000,28.083210,167,234,3,169.00,229.30
2016-08-29,3104.307692,5047.583159,67,16578,13,75.40,13501.80
2016-08-30,8993.500000,8977.500000,16,17971,2,913.75,17073.25
2016-08-31,128.736842,57.482118,44,236,19,50.30,200.00
2016-09-01,4343.068966,11892.752963,38,67099,58,125.50,34508.20


In [1069]:
pt = pd.pivot_table(df, index=['day'], values=["local time","remote time"], aggfunc=[nonzeromean, nonzerostd, nonzeromin, nonzeromax, np.count_nonzero, q05, q95])
pt

nonzeromean                nonzerostd               nonzeromin  \
            local time   remote time  local time   remote time local time   
day                                                                         
2016-08-25  202.255814  18301.560000  837.262735  14678.177484        6.0   
2016-08-26  413.966667  11198.733333  497.590359  24206.707375       16.0   
2016-08-27  122.785714   1763.000000  332.346243   2133.933962        1.0   
2016-08-28   58.000000    138.000000   29.810513      2.449490       29.0   
2016-08-29  135.714286   3940.600000  184.199205   5480.191205        1.0   
2016-08-30   15.000000  17957.000000    1.000000      0.000000       14.0   
2016-08-31   64.500000    167.200000   22.632941     10.609430       32.0   
2016-09-01   28.578947   4570.090909   63.752247  12170.716476        1.0   
2016-09-02  343.043478   1748.714286  781.693951   5389.325633        1.0   
2016-09-05   24.384615   3407.733333   18.743362   4459.499127        8.0   
2016-09-06  122.750000   1974.384615  329.557425   4553.581773        7.0   
2016-09-07   78.857143   8344.500000   77.334946  11017.561455        1.0   
2016-09-08         NaN     93.000000         NaN      0.000000        NaN   
2016-09-09   52.823529   9110.926829   90.594270  12852.984314        1.0   
2016-09-12   70.375000   3294.345679  102.881328   4237.770714        1.0   
2016-09-13         NaN    132.500000         NaN      0.500000        NaN   

                       nonzeromax             count_nonzero              \
           remote time local time remote time    local time remote time   
day                                                                       
2016-08-25         190     5574.0       45281            43          25   
2016-08-26         154     1809.0       97115            30          30   
2016-08-27         151     1309.0        6791            14          13   
2016-08-28         135       99.0         141             3           3   
2016-08-29         144      534.0       16577             7          10   
2016-08-30       17957       16.0       17957             2           1   
2016-08-31         147      115.0         191            12          10   
2016-09-01         139      293.0       67099            19          55   
2016-09-02         140     3715.0       24671            23          21   
2016-09-05         155       77.0       14077            13          15   
2016-09-06         161     2257.0       25923            44          52   
2016-09-07         181      182.0       31469             7          12   
2016-09-08          93        NaN          93             0           1   
2016-09-09          38      583.0       46075            51          41   
2016-09-12         131      667.0       16351            48          81   
2016-09-13         132        NaN         133             0           2   

                  q05                    q95              
           local time remote time local time remote time  
day                                                       
2016-08-25       7.00     2198.80     299.70    44246.40  
2016-08-26      27.95      154.45    1489.10    66524.60  
2016-08-27       1.00      160.00     581.65     5439.80  
2016-08-28      30.70      135.30      93.70      140.70  
2016-08-29       1.00      153.00     453.30    14269.85  
2016-08-30      14.10    17957.00      15.90    17957.00  
2016-08-31      38.60      154.20     100.15      184.25  
2016-09-01       1.00      140.00      73.40    34947.40  
2016-09-02       2.50      144.00    1149.80     8019.00  
2016-09-05       9.20      157.80      57.80    13255.90  
2016-09-06      16.45      166.00     190.55     9448.95  
2016-09-07       1.60      194.20     175.70    29100.70  
2016-09-08       0.00       93.00       0.00       93.00  
2016-09-09       1.00       78.00     175.50    36706.00  
2016-09-12       1.00      139.00     173.00    11789.00  
2016-09-13       0.00      132.05       0.00      132.95

In [1072]:
# DO NOT DELETE!
# subset expressions, containing " np"
expressions = df['expression']
expr = [expression for expression in expressions if " np" in expression]
expr = np.unique(expr).tolist()
expr = [" np"]
pt = pd.pivot_table(df[df.expression.isin(expr)], index=['day'], values=["local time","remote time"], aggfunc=[nonzeromean, nonzerostd, nonzeromin, nonzeromax, np.count_nonzero, q05, q95])
pt

nonzeromean                nonzerostd               nonzeromin  \
            local time   remote time  local time   remote time local time   
day                                                                         
2016-08-25   70.586207  27606.642857   68.279606  13295.742039          7   
2016-08-26  455.750000  29258.500000  430.603283   1377.500000        105   
2016-08-28   58.000000    138.000000   29.810513      2.449490         29   
2016-08-29   67.000000           NaN    0.000000           NaN         67   
2016-08-31   71.777778    147.000000   21.217277      0.000000         44   
2016-09-01  150.500000    402.000000  142.500000     57.000000          8   
2016-09-02  471.000000   2767.666667  938.266629   3713.253769         32   
2016-09-05   49.333333   2685.000000   21.044925    926.030597         26   
2016-09-06   79.333333   5682.181818   38.396687   6895.554653         27   
2016-09-07   23.000000   6185.500000    0.000000    114.500000         23   
2016-09-09   98.400000   6658.444444  145.362902   8097.294776         11   
2016-09-12   96.600000   7504.666667  134.519664   4828.238246         17   

                       nonzeromax             count_nonzero              \
           remote time local time remote time    local time remote time   
day                                                                       
2016-08-25      2626.0        305     45281.0            29          14   
2016-08-26     27881.0       1499     30636.0             8           2   
2016-08-28       135.0         99       141.0             3           3   
2016-08-29         NaN         67         NaN             1           0   
2016-08-31       147.0        115       147.0             9           1   
2016-09-01       345.0        293       459.0             2           2   
2016-09-02       140.0       3715      8019.0            15           3   
2016-09-05      1378.0         77      3410.0             3           3   
2016-09-06       549.0        194     25923.0            24          11   
2016-09-07      6071.0         23      6300.0             2           2   
2016-09-09       580.0        583     27554.0            15           9   
2016-09-12       705.0        667     16351.0            20          12   

                  q05                    q95              
           local time remote time local time remote time  
day                                                       
2016-08-25       8.00     6328.40     213.20    45134.10  
2016-08-26     121.10    28018.75    1185.40    30498.25  
2016-08-28      30.70      135.30      93.70      140.70  
2016-08-29      67.00        0.00      67.00        0.00  
2016-08-31      44.40      147.00     104.20      147.00  
2016-09-01      22.25      350.70     278.75      453.30  
2016-09-02      34.10      140.40    1926.50     7231.50  
2016-09-05      27.90     1566.90      73.80     3395.70  
2016-09-06      30.65      560.00     146.70    17427.00  
2016-09-07      23.00     6082.45      23.00     6288.55  
2016-09-09      11.00      785.60     378.60    20947.60  
2016-09-12      17.00      741.30     155.90    14346.80

# glacier timeseries

In [1052]:
y = df["total time"]
x = df["date"]
t = []
for i in range(len(x)):
    t.append(datetime.datetime.strptime(x[i][1:], "%Y/%m/%d %H:%M:%S"))
datemin = t[0] + timedelta(days=-1)
datemax = t[len(t) - 1] + timedelta(days=1)
from matplotlib import dates
days = dates.DayLocator()
hours = dates.HourLocator(interval=3)
dfmt = dates.DateFormatter('              %b %d')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.xaxis.set_major_locator(days)
ax.xaxis.set_major_formatter(dfmt)
ax.xaxis.set_minor_locator(hours)
ax.set_xlim(datemin, datemax)
ax.set_ylabel('Response time (total)')
ax.grid(True)
ax.plot_date(t, y, linewidth=2)
plt.show()

# Daily distribution

In [1053]:
pt.ix[pt.index[0]]

nonzeromean    local time       475.000000
               remote time     6458.727273
nonzerostd     local time      1425.882183
               remote time     3770.510843
nonzeromin     local time         6.000000
               remote time      190.000000
nonzeromax     local time      5574.000000
               remote time    12589.000000
count_nonzero  local time        14.000000
               remote time       11.000000
q05            local time         6.650000
               remote time     1141.000000
q95            local time      2427.350000
               remote time    12462.500000
Name: 2016-08-25 00:00:00, dtype: float64

In [1073]:
dat = {}
dat['time'] = pt.index
dat_mean_loc = []
dat_std_loc = []
dat_min_loc = []
dat_max_loc = []
dat_num_loc = []
dat_q10_loc = []
dat_q90_loc = []
dat_mean_rem = []
dat_std_rem = []
dat_min_rem = []
dat_max_rem = []
dat_num_rem = []
dat_q10_rem = []
dat_q90_rem = []
for date in pt.index:
    dat_mean_loc.append(pt.ix[date][0])
    dat_mean_rem.append(pt.ix[date][1])
    dat_std_loc.append(pt.ix[date][2])
    dat_std_rem.append(pt.ix[date][3])
    dat_min_loc.append(pt.ix[date][4])
    dat_min_rem.append(pt.ix[date][5])
    dat_max_loc.append(pt.ix[date][6])
    dat_max_rem.append(pt.ix[date][7])
    dat_num_loc.append(pt.ix[date][8])
    dat_num_rem.append(pt.ix[date][9])
    dat_q10_loc.append(pt.ix[date][10])
    dat_q10_rem.append(pt.ix[date][11])
    dat_q90_loc.append(pt.ix[date][12])
    dat_q90_rem.append(pt.ix[date][13])
dat['mean_loc'] = dat_mean_loc
dat['std_loc'] = dat_std_loc
dat['min_loc'] = dat_min_loc
dat['max_loc'] = dat_max_loc
dat['num_loc'] = dat_num_loc
dat['Q05_loc'] = dat_q10_loc
dat['Q95_loc'] = dat_q90_loc
dat['mean_rem'] = dat_mean_rem
dat['std_rem'] = dat_std_rem
dat['min_rem'] = dat_min_rem
dat['max_rem'] = dat_max_rem
dat['num_rem'] = dat_num_rem
dat['Q05_rem'] = dat_q10_rem
dat['Q95_rem'] = dat_q90_rem
#dat
#pd.DataFrame(dat,columns=["date","mean_loc","std_loc","min_loc","max_loc","Q05_loc","Q95_loc","mean_rem","std_rem","min_rem","max_rem","Q05_rem","Q95_rem"])

In [1074]:
#error bars with asymmetric bounds defined by quantiles
datemin = t[0] + timedelta(days=-1)
datemax = t[len(t) - 1] + + timedelta(days=1)
days = dates.DayLocator()
hours = dates.HourLocator(interval=3)
dfmt = dates.DateFormatter('              %b %d')
opacity = 0.5
fig = plt.figure()
# local time response
ax = fig.add_subplot(311)
ax.xaxis.set_major_locator(days)
ax.xaxis.set_major_formatter(dfmt)
ax.xaxis.set_minor_locator(hours)
ax.set_xlim(datemin, datemax)
ax.set_ylabel('Response time (local)')
ax.set_title('Local time response')
ax.grid(True)
ax.plot(dat['time'],dat['mean_loc'],color='blue',alpha=opacity)
ax.fill_between(dat['time'],dat['Q95_loc'],dat['Q05_loc'],color='blue',alpha=opacity)
ax.errorbar(x=dat['time'],y=dat['mean_loc'],color='b',yerr=[np.asarray(dat['mean_loc'])-np.asarray(dat['Q05_loc']),np.asarray(dat['Q95_loc'])-np.asarray(dat['mean_loc'])])
# remote time response
ax = fig.add_subplot(312)
ax.xaxis.set_major_locator(days)
ax.xaxis.set_major_formatter(dfmt)
ax.xaxis.set_minor_locator(hours)
ax.set_xlim(datemin, datemax)
ax.set_ylabel('Response time (remote)')
ax.set_title('Remote time response')
ax.grid(True)
ax.plot(dat['time'],dat['mean_rem'],color='red',alpha=opacity)
ax.fill_between(dat['time'],dat['Q95_rem'],dat['Q05_rem'],color='red',alpha=opacity)
ax.errorbar(x=dat['time'],y=dat['mean_rem'],color='r',yerr=[np.asarray(dat['mean_rem'])-np.asarray(dat['Q05_rem']),np.asarray(dat['Q95_rem'])-np.asarray(dat['mean_rem'])])
# histogram (number of completions)
ax = fig.add_subplot(313)
ax.xaxis.set_major_locator(days)
ax.xaxis.set_major_formatter(dfmt)
ax.xaxis.set_minor_locator(hours)
ax.set_xlim(datemin, datemax)
ax.set_ylabel('Number of completions (local)')
ax.set_title('Number of completions')
ax.grid(True)
ax.plot_date(dat['time'],dat['num_loc'],alpha=opacity,ls='-',color='b',label='Local')
ax.plot_date(dat['time'],dat['num_rem'],alpha=opacity,ls='-',color='r',label='Remote')
plt.legend()
plt.show()

# Daily distribution histogram

In [692]:
df.groupby(["localization"]).hist()
plt.show()

In [693]:
from pandas.tools.plotting import scatter_matrix
scatter_matrix(df, alpha=0.5,diagonal='kde')
plt.show()

# Fitting distribution

In [1056]:
# grouping and aggregating data by dates
grouped = df.groupby(["day"])
df1 = grouped.aggregate(lambda x:tuple(x))
df1["total time"]

day
2016-08-25    (0, 30370, 45586, 100, 47, 46, 31072, 25382, 3...
2016-08-26    (455, 18182, 186, 603, 521, 151, 110, 57, 2183...
2016-08-27    (365, 193, 183, 179, 181, 58, 6792, 151, 3251,...
2016-08-28                                      (187, 167, 234)
2016-08-29    (81, 534, 1136, 198, 165, 173, 144, 16578, 631...
2016-08-30                                          (17971, 16)
2016-08-31    (176, 171, 163, 164, 164, 164, 168, 192, 51, 6...
2016-09-01    (167, 159, 142, 143, 139, 141, 141, 1224, 142,...
2016-09-02    (185, 175, 110, 65, 72, 24671, 140, 61, 62, 29...
2016-09-05    (181, 172, 159, 165, 199, 37, 734, 1848, 203, ...
2016-09-06    (171, 194, 2257, 163, 179, 172, 165, 174, 201,...
2016-09-07    (182, 159, 161, 6323, 6094, 31469, 2027, 205, ...
2016-09-08                                                (93,)
2016-09-09    (183, 168, 20, 16837, 102, 7069, 8, 1105, 6529...
2016-09-12    (168, 173, 153, 173, 156, 8897, 8458, 8734, 86...
2016-09-13                          

In [1057]:
import statsmodels.api as sm
def QQplot(df,dist_name="norm", value_name="local time", L = 1,line = 'q'):
    grouped = df.groupby(["day"])
    days = grouped.aggregate(lambda x:tuple(x))[value_name]
    h = min(L,len(days.index))
    fig = plt.figure()
    i = 0
    for day in days[-h:]:
        ax = fig.add_subplot(h,1,i+1)
        ax.set_title(dist_name + days.index[i].strftime("%Y/%m/%d"))
        y = np.asarray(day)
        y = y[y>0]
        dist = getattr(scipy.stats, dist_name)
        sm.graphics.qqplot(y, dist=dist, fit = True, line = line, ax = ax)
        i+=1
    return fig

In [1058]:
QQplot(df,"pareto","total time",2)
plt.show()

In [1059]:
import statsmodels.api as sm
def QQplotXY(df,dist_name="norm", value_name="local time", L = 1,line = 'q'):
    grouped = df.groupby(["day"])
    days = grouped.aggregate(lambda x:tuple(x))[value_name]
    h = max(min(len(days.index)-1,L),1)
    fig = plt.figure()
    i = 0
    dayX = days[-h-1]
    dayY = days[-h]
    ax = fig.add_subplot(1,1,1)
    ax.set_title(dist_name + days.index[-h].strftime("%Y/%m/%d"))
    x = np.asarray(dayX)
    x = x[x>0]
    y = np.asarray(dayY)
    y = y[y>0]
    length = min(len(x),len(y))
    x = x[:length]
    y = y[:length]
    dist = getattr(scipy.stats, dist_name)
    qq_x = sm.ProbPlot(x, dist = dist, fit=True)
    qq_y = sm.ProbPlot(y, dist = dist, fit=True)
    qq_x.qqplot(other=qq_y, line=line, ax = ax)
    return fig

In [1060]:
QQplotXY(df,"expon","local time",1)
plt.show()

C:\Users\Denis.Stepanov\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
C:\Users\Denis.Stepanov\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\core\_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


ValueError: zero-size array to reduction operation minimum which has no identity

In [1061]:
def fitDist(df, dist_name="norm", value_name="local time", conf = 0.95):
    grouped = df.groupby(["day"])
    days = grouped.aggregate(lambda x:tuple(x))[value_name]
    columns = ['num','sample_mean','sample_std','sample_Q05','sample_Q95','dist_mean','dist_std','dist_Q05','dist_Q95','a','loc','scale']
    w, h = len(columns), len(days.index) 
    data = [[0 for x in range(w)] for y in range(h)]
    tbl = pd.DataFrame(data,index=days.index,columns=columns)
    i = 0
    for day in days:
        y = np.asarray(day)
        y = y[y>0]
        dist = getattr(scipy.stats, dist_name)
        param = dist.fit(y)
        Q = dist.interval(conf,*param[:-2],loc=param[-2],scale=param[-1])
        tbl["num"][i] = np.count_nonzero(y)
        tbl["sample_mean"][i] = np.mean(y)
        tbl["sample_std"][i] = np.std(y)
        tbl["sample_Q05"][i] = q05(y)
        tbl["sample_Q95"][i] = q95(y)
        if np.count_nonzero(y)>2:
            if abs(dist.mean(*param[:-2],loc=param[-2],scale=param[-1])) < 1e10:
                tbl["dist_mean"][i] = dist.mean(*param[:-2],loc=param[-2],scale=param[-1])
            if abs(dist.std(*param[:-2],loc=param[-2],scale=param[-1])) < 1e10:
                tbl["dist_std"][i] = dist.std(*param[:-2],loc=param[-2],scale=param[-1])
            tbl["dist_Q05"][i] = Q[0]
            tbl["dist_Q95"][i] = Q[1]
        if len(param) == 3:
            tbl["a"][i] = param[-3]
        tbl["loc"][i] = param[-2]
        tbl["scale"][i] = param[-1]
        i+=1
    return tbl


In [1062]:
fitDist(df,"expon","local time")
#%save -f fitDist_file 957

C:\Users\Denis.Stepanov\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
C:\Users\Denis.Stepanov\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\core\_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


ValueError: zero-size array to reduction operation minimum which has no identity

In [1063]:
dist_name = 'expon'
conf = 0.95
# (1) aggregate by day
grouped = df.groupby(["day"])
days = grouped.aggregate(lambda x:tuple(x))["local time"]
columns = ['num','sample_mean','sample_std','sample_Q05','sample_Q95','dist_mean','dist_std','dist_Q05','dist_Q95','a','loc','scale']
w, h = len(columns), len(days.index) 
data = [[0 for x in range(w)] for y in range(h)]
tbl = pd.DataFrame(data,index=days.index,columns=columns)
i = 0
for day in days:
    y = np.asarray(day)
    y = y[y>0]
    dist = getattr(scipy.stats, dist_name)
    param = dist.fit(y)
    Q = dist.interval(conf,*param[:-2],loc=param[-2],scale=param[-1])
    tbl["num"][i] = np.count_nonzero(y)
    tbl["sample_mean"][i] = np.mean(y)
    tbl["sample_std"][i] = np.std(y)
    tbl["sample_Q05"][i] = q05(y)
    tbl["sample_Q95"][i] = q95(y)
    if np.count_nonzero(y)>2:
        if abs(dist.mean(*param[:-2],loc=param[-2],scale=param[-1])) < 1e10:
            tbl["dist_mean"][i] = dist.mean(*param[:-2],loc=param[-2],scale=param[-1])
        if abs(dist.std(*param[:-2],loc=param[-2],scale=param[-1])) < 1e10:
            tbl["dist_std"][i] = dist.std(*param[:-2],loc=param[-2],scale=param[-1])
        tbl["dist_Q05"][i] = Q[0]
        tbl["dist_Q95"][i] = Q[1]
    if len(param) == 3:
        tbl["a"][i] = param[-3]
    tbl["loc"][i] = param[-2]
    tbl["scale"][i] = param[-1]
    i+=1
tbl

C:\Users\Denis.Stepanov\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
C:\Users\Denis.Stepanov\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\core\_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


ValueError: zero-size array to reduction operation minimum which has no identity

In [1064]:
dist_name = 'lognorm'
conf = 0.95
# (1) aggregate by day
grouped = df.groupby(["day"])
days = grouped.aggregate(lambda x:tuple(x))["local time"]
columns = ['num','sample_mean','sample_std','sample_Q05','sample_Q95','dist_mean','dist_std','dist_Q05','dist_Q95','a','loc','scale']
w, h = len(columns), len(days.index) 
data = [[0 for x in range(w)] for y in range(h)]
tbl = pd.DataFrame(data,index=days.index,columns=columns)
i = 0
for day in days:
    y = np.asarray(day)
    y = y[y>0]
    dist = getattr(scipy.stats, dist_name)
    param = dist.fit(y)
    print(param)

(1.9252473379468471, 5.8316606096962236, 26.295803060852609)
(2.5568749453365571, 15.997455903558599, 1.7836639613511429)
(4.9349940394185765, 0.99999999999388467, 0.23101252293377067)
(8.0569059193237536, 28.999999999998519, 0.0011636791196289607)
(8.7871867717558167, 0.99999999999999989, 163.67375926085992)
(9.0244465933083085, 13.999999999999996, 0.054685394929317163)
(7.9510925537695787, 31.999999999999993, 4.5934846371622022)
(8.6304979033785969, 0.99999999999999989, 1.22575493152004)
(1.7039234704117634, -1.0740674024363002, 81.702147610142504)
(1.3365171003767324, 7.5015938067251371, 8.3119868826792978)
(1.074430897336597, 3.633514903756712, 53.618981177217975)
(2.5800023787353568, 0.99813005221429019, 1.5000782818120664)


C:\Users\Denis.Stepanov\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
C:\Users\Denis.Stepanov\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\core\_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


ValueError: zero-size array to reduction operation minimum which has no identity

In [703]:
dist_name = 'cauchy'
conf = 0.95
# (1) aggregate by day
grouped = df.groupby(["day"])
days = grouped.aggregate(lambda x:tuple(x))["local time"]
columns = ['num','sample_mean','sample_std','sample_Q05','sample_Q95','dist_mean','dist_std','dist_Q05','dist_Q95','a','loc','scale']
w, h = len(columns), len(days.index) 
data = [[0 for x in range(w)] for y in range(h)]
tbl = pd.DataFrame(data,index=days.index,columns=columns)
i = 0
for day in days:
    y = np.asarray(day)
    y = y[y>0]
    dist = getattr(scipy.stats, dist_name)
    param = dist.fit(y)
    print(len(param))

2
2
2
2
2
2
2
2
2
2
2
2


In [400]:
# convert tuple to array with non-zero elements
y = np.asarray(df1["local time"][0])
y = y[y>0]

In [401]:
y

array([  15,  305,  100,   47,   46,   10,    7,    8,    9,    9,    8,
         76,   41,   71,   74,   59,  110,   52,   51,   43,  155,   61,
       5574,    9,  733,   51,  252,  109,  104,   50,   10,    7,    6,
          7,   79,   31,   25,   25,   60,   37,   47,   16,  108])

In [382]:
y = np.log(y)
y

array([ 5.6454469 ,  2.77258872,  5.22574667,  6.4019172 ,  6.25575004,
        5.01727984,  4.70048037,  4.04305127,  3.8286414 ,  3.66356165,
        3.52636052,  3.13549422,  7.3125535 ,  4.65396035,  7.29776828,
        4.90527478,  5.53338949,  5.37063803,  5.56068163,  5.10594547,
        5.23644196,  7.1592919 ,  6.91968385,  6.83303173,  5.31811999,
        7.50052949,  4.40671925,  5.01727984,  6.00141488,  5.17614973])

In [383]:
plt.hist(y)
plt.show()

In [551]:
fitted_dist = {}
dist_names = ['expon','gamma', 'pareto','cauchy']
dist_param = []
dist_interval = []
conf = 0.95

for dist_name in dist_names:
    dist = getattr(scipy.stats, dist_name)
    param = dist.fit(y)
    dist_param.append(param)
    dist_interval.append(dist.interval(conf, *param[:-2], loc=param[-2], scale=param[-1]))

fitted_dist["dist"] = dist_names
fitted_dist["param"] = dist_param
fitted_dist["confInt"] = dist_interval
fitted_dist

{'confInt': [(10.96662401806633, 729.65180065306322),
  (6.0000019075520994, 4895.2840310913389),
  (6.3046278921476038, 8062.9977596981707),
  (-299.75004951738038, 380.83807334841856)],
 'dist': ['expon', 'gamma', 'pareto', 'cauchy'],
 'param': [(5.9999999891181854, 196.17117058593755),
  (0.1766580719190593, 5.9999999999999991, 3480.3409891048518),
  (0.51225916912224678, -0.012524679722765823, 6.0125246686674734),
  (40.544011915519114, 26.78172345705077)]}

In [403]:
fitted_dist["dist"][0]

'expon'

In [406]:
fitted_dist["param"][0]

(5.9999999891181854, 196.17117058593755)

In [461]:
i = 3
y = np.asarray(df1["local time"][i])
y = y[y>0]

import pylab
import scipy.stats as stats

stats.probplot(y, dist=fitted_dist["dist"][i], sparams=fitted_dist["param"][i], plot=pylab)
pylab.show()

In [460]:
import statsmodels.api as sm
i = 3
y = np.asarray(df1["local time"][i])
y = y[y>0]

sm.qqplot(y, dist=fitted_dist["dist"][i], fit = True, line = 'q')
pylab.show()

In [470]:
# compare quantiles
import statsmodels.api as sm
x = np.random.normal(loc=8.00, scale=3.5, size=1000)
y = np.random.normal(loc=8.00, scale=3.5, size=1000)
pp_x = sm.ProbPlot(x, fit=True)
pp_y = sm.ProbPlot(y, fit=True)
# compare x quantiles to y quantiles
fig3 = pp_x.qqplot(other=pp_y, line='q')
plt.show()

In [359]:
import matplotlib.pyplot as plt
import scipy
import scipy.stats
size = len(y)
x = scipy.arange(size)
h = plt.hist(y, bins='auto',color='w')

dist_names = ['expon','gamma', 'pareto']

for dist_name in dist_names:
    dist = getattr(scipy.stats, dist_name)
    param = dist.fit(y)
    pdf_fitted = dist.pdf(x, *param[:-2], loc=param[-2], scale=param[-1]) * size
    plt.plot(pdf_fitted, label=dist_name)
    plt.xlim(0,np.max(y))
plt.legend(loc='upper right')
plt.show()

In [ ]:
# list of scipy probability distributions
dist_names = [ 'alpha', 'anglit', 'arcsine', 'beta', 'betaprime', 'bradford', 'burr', 'cauchy', 'chi', 'chi2', 'cosine', 'dgamma', 'dweibull', 'erlang', 'expon', 'exponweib', 'exponpow', 'f', 'fatiguelife', 'fisk', 'foldcauchy', 'foldnorm', 'frechet_r', 'frechet_l', 'genlogistic', 'genpareto', 'genexpon', 'genextreme', 'gausshyper', 'gamma', 'gengamma', 'genhalflogistic', 'gilbrat', 'gompertz', 'gumbel_r', 'gumbel_l', 'halfcauchy', 'halflogistic', 'halfnorm', 'hypsecant', 'invgamma', 'invgauss', 'invweibull', 'johnsonsb', 'johnsonsu', 'ksone', 'kstwobign', 'laplace', 'logistic', 'loggamma', 'loglaplace', 'lognorm', 'lomax', 'maxwell', 'mielke', 'nakagami', 'ncx2', 'ncf', 'nct', 'norm', 'pareto', 'pearson3', 'powerlaw', 'powerlognorm', 'powernorm', 'rdist', 'reciprocal', 'rayleigh', 'rice', 'recipinvgauss', 'semicircular', 't', 'triang', 'truncexpon', 'truncnorm', 'tukeylambda', 'uniform', 'vonmises', 'wald', 'weibull_min', 'weibull_max', 'wrapcauchy']